# Notes

files or dataframes can be opened with this function:</br>
open_in_excel(df, output_csv_path=r'd:\output.csv', excel_path=None, encoding='utf-8', verbose=True)

# Load the Code

## Packages, Settings, and External Links

### Imports and Paths

In [98]:
import os
import re
import chardet
import string
import platform
import subprocess
import numpy as np

import pandas as pd
from IPython.display import display
from dataframe_handler import DataFrameHandler

import nltk
from nltk import ngrams, word_tokenize
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
from autocorrect import Speller
from collections import Counter
import openpyxl #Used by Pandas to open Excel files
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt



read_file = r'C:\Users\Windows\Downloads\archive (7)\netflix1.csv'

### Qualitative Analysis Lists and Settings

In [99]:
# Stopword lisit
external_stopword_list = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\stopwords_seo.csv"

# Contractions dictionary
external_contractions_dictionary = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\contractions_list.csv"

# External search and replace file
custom_search_and_replace_dictionary = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\custom search-and-replace dictionary.csv"

### Set Pandas Display Options

In [100]:
# Set options to display more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (if applicable)
pd.set_option('display.max_colwidth', None) # Show full column width
pd.set_option('display.width', 1000)        # Set the width of the display

## Load Functions

### Misc Support Functions

In [101]:
# Load a 2-column csv file as a pandas dataframe and return a dictionary 

def load_csv_to_dict(file_path, key_column, value_column):
    """
    Load a CSV file and create a dictionary from specified key and value columns.

    Parameters:
    file_path (str): The path to the CSV file.
    key_column (str): The name of the column to use as keys in the dictionary.
    value_column (str): The name of the column to use as values in the dictionary.

    Returns:
    dict: A dictionary with keys from the key_column and values from the value_column.
    """
    df = pd.read_csv(file_path)
    
    # Convert NaN values to empty strings
    df[value_column] = df[value_column].apply(lambda x: '' if pd.isna(x) else str(x))
    
    return df.set_index(key_column)[value_column].to_dict()

def ensure_string(text, string_length_threshold):
    """
    Ensure the input is converted to a string representation.

    Parameters:
    text: The input to be checked and converted to a string if necessary.
    string_length_threshold: The minimum length of the string. If the length of the string is less than this threshold, return an empty string.

    Returns:
    str: The input converted to a string or an empty string if the length is less than the threshold or conversion fails.
    """
    if pd.isnull(text):
        return ""
    elif isinstance(text, str):
        text = text.strip()  # Strip whitespace from both ends
        if len(text) < string_length_threshold:
            return ""
        return text
    
    try:
        # Attempt to convert non-string types to string
        text = str(text).strip()
        if len(text) < string_length_threshold:
            return ""
        return text
    except (ValueError, TypeError):
        return ""
    
def clean_whitespace(text):
    # Remove leading and trailing whitespace
    cleaned_text = text.strip()
    
    # Replace multiple spaces with a single space
    cleaned_text = ' '.join(cleaned_text.split())
    
    return cleaned_text


def save_dataframe_with_incremented_filename(file_path):
    """
    Check if a filename already exists. If it does, it returns an incremented filename.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    file_path (str): The initial file path for the CSV file.
    """
    base, extension = os.path.splitext(file_path)
    counter = 1

    # Check if the file already exists
    while os.path.exists(file_path):
        # Increment the file name
        file_path = f"{base}_{counter}{extension}"
        counter += 1

    return file_path

### Load to dataframe

In [102]:
import pandas as pd
import os
import chardet

def detect_encoding(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The file at {file_path} does not exist.")
    
    try:
        with open(file_path, 'rb') as file:
            result = chardet.detect(file.read(10000))
        return result['encoding']
    except IOError as e:
        raise IOError(f"An error occurred while reading the file: {e}")

def detect_delimiter(file_path, sample_size=1000):
    delimiters = [',', '\t', ';', '|', ' ']
    delimiter_counts = {delim: 0 for delim in delimiters}
    
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        sample = file.read(sample_size)
    
    for delim in delimiters:
        delimiter_counts[delim] = sample.count(delim)
    
    # Handle the case where multiple delimiters have similar counts
    most_common_delim = max(delimiter_counts, key=delimiter_counts.get)
    
    return most_common_delim

def read_file(file_path, encoding='utf-8', sheet_name=None, force_plain_text=False):
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()[1:]
    
    try:
        if file_extension in ['csv']:
            print(f"Loading {file_path} as CSV with default comma delimiter")
            return pd.read_csv(file_path, encoding=encoding, on_bad_lines='skip')
        elif file_extension in ['tsv']:
            print(f"Loading {file_path} as TSV with tab delimiter")
            return pd.read_csv(file_path, delimiter='\t', encoding=encoding, on_bad_lines='skip')
        elif file_extension in ['txt', 'log']:
            if force_plain_text:
                print(f"Loading {file_path} as plain text")
                with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
                    content = file.read()
                return pd.DataFrame({'content': [content]})
            else:
                delimiter = detect_delimiter(file_path)
                print(f"Detected delimiter for {file_path}: '{delimiter}'")
                return pd.read_csv(file_path, delimiter=delimiter, encoding=encoding, on_bad_lines='skip')
        elif file_extension in ['xls', 'xlsx']:
            return pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
        elif file_extension == 'json':
            return pd.read_json(file_path, encoding=encoding)
        elif file_extension == 'parquet':
            return pd.read_parquet(file_path)
        elif file_extension == 'hdf':
            return pd.read_hdf(file_path)
        elif file_extension == 'feather':
            return pd.read_feather(file_path)
        else:
            raise ValueError("Unsupported file type.")
    except (UnicodeDecodeError, pd.errors.ParserError) as e:
        detected_encoding = detect_encoding(file_path)
        try:
            if file_extension in ['csv', 'json', 'txt', 'log']:
                if force_plain_text:
                    print(f"Reattempting with detected encoding '{detected_encoding}' as plain text")
                    with open(file_path, 'r', encoding=detected_encoding, errors='ignore') as file:
                        content = file.read()
                    return pd.DataFrame({'content': [content]})
                else:
                    delimiter = detect_delimiter(file_path)
                    print(f"Reattempting with detected encoding '{detected_encoding}' and delimiter '{delimiter}'")
                    return pd.read_csv(file_path, delimiter=delimiter, encoding=detected_encoding, on_bad_lines='skip')
            else:
                raise RuntimeError("Unsupported file type for encoding detection.")
        except Exception as e:
            raise RuntimeError(f"An error occurred while loading the file: {e}")

def load_to_dataframe(file_or_folder_path, file_type=None, sheet_name=None, force_plain_text=False):
    if os.path.isdir(file_or_folder_path):
        if file_type is None:
            raise ValueError("File type must be specified when a folder is provided.")
        
        dfs = []
        columns_set = set()  # To keep track of all columns across files
        
        for root, _, files in os.walk(file_or_folder_path):
            for file in files:
                if file.lower().endswith(file_type.lower()):
                    file_path = os.path.join(root, file)
                    print(f"Processing file: {file_path}")
                    try:
                        df = read_file(file_path, sheet_name=sheet_name, force_plain_text=force_plain_text)
                        columns_set.update(df.columns)
                        dfs.append(df)
                    except Exception as e:
                        print(f"Warning: Could not process file {file_path}: {e}")
                        try:
                            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                                content = file.read()
                            df = pd.DataFrame({'content': [content]})
                            dfs.append(df)
                        except Exception as fallback_e:
                            print(f"Fallback failed for file {file_path}: {fallback_e}")

        if not dfs:
            raise ValueError("No valid files found in the specified folder.")
        
        # Standardize columns across all DataFrames
        standardized_dfs = []
        for df in dfs:
            # Add missing columns
            missing_cols = columns_set - set(df.columns)
            for col in missing_cols:
                df[col] = pd.NA
            # Reorder columns
            df = df[list(columns_set)]
            standardized_dfs.append(df)

        combined_df = pd.concat(standardized_dfs, ignore_index=True)
        return combined_df

    elif os.path.isfile(file_or_folder_path):
        return read_file(file_or_folder_path, sheet_name=sheet_name, force_plain_text=force_plain_text)
    
    else:
        raise ValueError(f"The path {file_or_folder_path} is neither a file nor a folder.")


### Dataframe Diagnostics

In [103]:
def dataframe_diagnostics(df: pd.DataFrame, encoding_check: bool = True) -> pd.DataFrame:
    """
    Provides diagnostics for potential issues in the DataFrame including missing values, incorrect types, encoding issues, etc.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to analyze.
    - encoding_check (bool): If True, checks for columns with potential encoding issues.

    Returns:
    - pd.DataFrame: A DataFrame summarizing potential issues with the data.
    """
    
    issues = []

    # Step 1: Check for missing values
    missing_vals = df.isnull().sum()
    missing_percent = (df.isnull().sum() / len(df)) * 100
    for col, count, percent in zip(df.columns, missing_vals, missing_percent):
        if count > 0:
            issues.append({
                'Column': col, 
                'Issue': f"Missing values: {count} ({percent:.2f}%)"
            })

    # Step 2: Check for potential data type issues
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if object column contains numeric values
            try:
                pd.to_numeric(df[col])
                issues.append({
                    'Column': col,
                    'Issue': 'Potential data type issue: Object column contains numeric values'
                })
            except:
                pass

    # Step 3: Check for constant columns (columns with a single unique value)
    for col in df.columns:
        if df[col].nunique() == 1:
            issues.append({
                'Column': col, 
                'Issue': 'Constant column: Only one unique value'
            })

    # Step 4: Check for columns with high cardinality (many unique values)
    for col in df.columns:
        if df[col].nunique() > 0.9 * len(df):
            issues.append({
                'Column': col, 
                'Issue': 'High cardinality: Large number of unique values'
            })

    # Step 5: Check for potential encoding issues (non-UTF8 characters or odd encodings)
    if encoding_check:
        for col in df.select_dtypes(include=['object']).columns:
            try:
                df[col].apply(lambda x: x.encode('utf-8').decode('utf-8') if isinstance(x, str) else x)
            except UnicodeDecodeError:
                issues.append({
                    'Column': col,
                    'Issue': 'Potential encoding issue: Non-UTF8 characters detected'
                })

    # Step 6: Check for duplicate columns
    duplicate_columns = df.columns[df.T.duplicated()]
    for col in duplicate_columns:
        issues.append({
            'Column': col,
            'Issue': 'Duplicate column: Identical to another column'
        })

    # Step 7: Check for potential outliers (using basic IQR method)
    for col in df.select_dtypes(include=[np.number]).columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        if outliers > 0:
            issues.append({
                'Column': col,
                'Issue': f"Potential outliers: {outliers} rows outside IQR"
            })

    # Convert issues list to DataFrame for easier viewing
    issues_df = pd.DataFrame(issues)

    if issues_df.empty:
        print("No significant issues detected in the DataFrame.")
    else:
        display(issues_df)

    return issues_df

### Open in Excel

In [104]:
def open_csv_in_excel(file_path: str, excel_path=None):
    """
    Opens a CSV file in Microsoft Excel or other default programs based on the platform.
    
    Args:
        file_path (str): The path to the CSV file that needs to be opened.
        excel_path (str): Optional manual path to the Excel executable.
    
    Returns:
        None
    """
    if platform.system() == 'Windows':
        # Step 1: Check if Excel path is provided
        if not excel_path:
            excel_paths = [
                r"C:\Program Files (x86)\Microsoft Office\root\Office16\EXCEL.EXE",
                r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE"
            ]
            excel_program_path = None

            # Step 2: Check common Excel paths
            for path in excel_paths:
                if os.path.exists(path):
                    excel_program_path = path
                    break

            # Step 3: If Excel is not found in the predefined paths, search for it
            if not excel_program_path:
                directories_to_search = [
                    r"C:\Program Files",
                    r"C:\Program Files (x86)",
                    r"C:\\"
                ]
                excel_program_path = find_excel_exe(directories_to_search)

        else:
            excel_program_path = excel_path

        # Step 4: Open file with Excel if path is found
        if excel_program_path:
            subprocess.Popen([excel_program_path, file_path])
        else:
            print("Excel cannot be found on the system. Opening file with the default CSV viewer.")
            open_file_with_default_program(file_path)

    elif platform.system() == 'Darwin':  # macOS
        subprocess.call(['open', file_path])
    elif platform.system() == 'Linux':  # Linux
        subprocess.call(['xdg-open', file_path])
    else:
        print("Unsupported operating system. Unable to open the file.")


def find_excel_exe(directories: list) -> str:
    """
    Searches for EXCEL.EXE in the given directories.
    
    Args:
        directories (list): List of directories to search for the Excel executable.
    
    Returns:
        str: The full path to the Excel executable, or None if not found.
    """
    for directory in directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.lower() == 'excel.exe':
                    return os.path.join(root, file)
    return None


def open_file_with_default_program(file_path):
    """
    Opens a file using the default program associated with the file type.
    
    Args:
        file_path (str): The path to the file.
    
    Returns:
        None
    """
    try:
        if platform.system() == 'Windows':
            os.startfile(file_path)
        elif platform.system() == 'Darwin':  # macOS
            subprocess.call(['open', file_path])
        elif platform.system() == 'Linux':  # Linux
            subprocess.call(['xdg-open', file_path])
    except Exception as e:
        print(f"Error opening file: {e}")


def open_in_excel(input_data, output_csv_path='output.csv', excel_path=None, encoding='utf-8', verbose=True):
    """
    Opens a CSV file in Excel or the default program. If a DataFrame is provided, it saves it as a CSV first.
    
    Args:
        input_data (str or pd.DataFrame): Either a file path to a CSV or a Pandas DataFrame.
        output_csv_path (str): The file path to save the DataFrame if one is provided.
        excel_path (str): Optional path to Excel executable.
        encoding (str): Encoding to use when saving the CSV file.
        verbose (bool): If True, provides more detailed output on the process.
    
    Returns:
        None
    """
    file_to_open = None

    # Check if input_data is a DataFrame
    if isinstance(input_data, pd.DataFrame):
        if verbose:
            print(f"Saving DataFrame to CSV: {output_csv_path}")
        try:
            input_data.to_csv(output_csv_path, index=False, encoding=encoding)
            file_to_open = output_csv_path
        except Exception as e:
            print(f"Error saving DataFrame as CSV: {e}")
            return

    elif isinstance(input_data, str):
        # Assume it's a file path
        if os.path.exists(input_data):
            file_to_open = input_data
        else:
            print(f"The provided file path does not exist: {input_data}")
            return
    else:
        print("Invalid input. Please provide a valid DataFrame or file path.")
        return

    # Prompt the user if they want to open the file in Excel
    if verbose:
        print(f"File written to: {file_to_open}")
    open_csv = input("Open file in Excel? (Y/N): ").strip().lower()
    
    if open_csv == 'y':
        open_csv_in_excel(file_to_open, excel_path=excel_path)
    else:
        print("File not opened in Excel.")


### General Analysis

#### Overall Summary

In [105]:
def overall_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Get a comprehensive summary of the DataFrame including overall metrics.

    Parameters:
        df (pd.DataFrame): The DataFrame to summarize.

    Returns:
        pd.DataFrame: Summary including overall metrics about the DataFrame.
    """
    if df is None:
        raise ValueError("DataFrame is not set.")

    # Compute overall metrics
    total_rows = df.shape[0]
    total_columns = df.shape[1]
    total_memory_kb = (df.memory_usage(deep=True).sum() / 1024).round(1)
    total_missing_values = df.isna().sum().sum()
    total_duplicates = df.duplicated().sum()
    total_unique_values = df.nunique().sum()
    total_non_null_values = df.notna().sum().sum()
    data_types_count = df.dtypes.value_counts()

    # Format the metrics for readability
    formatted_total_rows = f"{total_rows:,}"
    formatted_total_columns = f"{total_columns:,}"
    formatted_total_memory_kb = f"{total_memory_kb:,.1f}"
    formatted_total_missing_values = f"{total_missing_values:,}"
    formatted_total_duplicates = f"{total_duplicates:,}"
    formatted_total_unique_values = f"{total_unique_values:,}"
    formatted_total_non_null_values = f"{total_non_null_values:,}"

    # Format the data types count for human readability
    data_types_summary = "\n".join([f"{dtype}: {count:,}" for dtype, count in data_types_count.items()])

    # Create overall summary DataFrame
    overall_summary_df = pd.DataFrame({
        'Metric': ['Total Rows', 'Total Columns', 'Total Memory Usage (KB)', 'Total Missing Values', 
                   'Total Duplicates', 'Total Unique Values', 'Total Non-Null Values', 'Data Types Count'],
        'Value': [formatted_total_rows, formatted_total_columns, formatted_total_memory_kb, formatted_total_missing_values, 
                  formatted_total_duplicates, formatted_total_unique_values, formatted_total_non_null_values, data_types_summary]
    })

    return overall_summary_df

#### Column Summaries

In [106]:
def data_types_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Get the data types of each column along with additional metrics and return them as a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame for which to generate the data types summary.

    Returns:
        pd.DataFrame: Data types and various metrics for each column.
    """
    if df is None:
        raise ValueError("DataFrame is not set.")

    # Initialize the summary DataFrame
    summary_dict = {
        'Column Name': df.columns,
        'Data Type': df.dtypes,
        'Non-Null Count': df.notna().sum(),
        'Null Count': df.isna().sum(),
        'Total Count': [df.shape[0]] * len(df.columns),  # Ensure same length
        'Percentage Non-Null': (df.notna().sum() / df.shape[0] * 100).round(2),
        'Distinct Count': df.nunique(),
        'Memory Usage (KB)': (df.memory_usage(deep=True, index=False) / 1024).round(1)  # Round to 1 decimal place
    }

    # Add Most Frequent Value (Mode)
    def get_most_frequent(column: pd.Series):
        try:
            mode = column.mode()
            return mode.iloc[0] if not mode.empty else None
        except:
            return None

    summary_dict['Most Frequent Value'] = df.apply(get_most_frequent)

    # Add Unique Count (number of values that appear only once)
    def unique_count(column: pd.Series) -> int:
        return (column.value_counts() == 1).sum()
    
    summary_dict['Unique Count'] = df.apply(unique_count)

    # Add Empty String Count for string columns
    def empty_string_count(column: pd.Series) -> int:
        if pd.api.types.is_string_dtype(column):
            return column.str.strip().eq('').sum()
        return 0

    summary_dict['Empty String Count'] = df.apply(empty_string_count)

    # Create DataFrame from dictionary
    summary = pd.DataFrame(summary_dict)

    # Ensure the DataFrame's index length matches the number of columns
    assert len(summary) == len(df.columns), "Mismatch in DataFrame column lengths."

    return summary

#### Column Details

In [107]:
def numeric_metrics(series: pd.Series) -> pd.Series:
    """
    Calculate various metrics for a numeric series.

    Args:
        series (pd.Series): The numeric series to analyze.

    Returns:
        pd.Series: Various calculated metrics for the series.
    """
    # Calculate basic statistics
    mean = series.mean()
    median = series.median()
    min_val = series.min()
    max_val = series.max()
    range_val = max_val - min_val
    total_sum = series.sum()
    
    # Calculate dispersion metrics
    std_dev = series.std()
    variance = series.var()
    abs_mean_dev = (series - mean).abs().mean()
    cv = std_dev / mean if mean != 0 else float('inf')
    
    # Calculate quartiles and IQR
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    
    # Calculate outlier bounds
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Calculate mode(s)
    mode_values = series.mode()
    mode_value = mode_values[0] if not mode_values.empty else np.nan
    
    # Calculate additional metrics
    skewness = series.skew()
    kurtosis = series.kurtosis()
    num_outliers = ((series < lower_bound) | (series > upper_bound)).sum()
    missing_values = series.isna().sum()
    unique_values = series.nunique()

    # Format the metrics with thousands separators and up to two decimal places
    def format_number(value):
        if pd.isna(value):
            return 'NaN'
        return f"{value:,.2f}"
    
    # Compile all metrics into a pandas Series with logical grouping
    return pd.Series({
        # Basic Statistics
        'Mean': format_number(mean),
        'Median': format_number(median),
        'Min': format_number(min_val),
        'Max': format_number(max_val),
        'Range': format_number(range_val),
        'Sum': format_number(total_sum),

        # Dispersion
        'Std Dev': format_number(std_dev),
        'Variance': format_number(variance),
        'Absolute Mean Deviation': format_number(abs_mean_dev),
        'Coefficient of Variation (CV)': format_number(cv),

        # Distribution
        'Skewness': format_number(skewness),
        'Kurtosis': format_number(kurtosis),

        # Quartiles and IQR
        '1st Quartile (Q1)': format_number(q1),
        '3rd Quartile (Q3)': format_number(q3),
        'Interquartile Range (IQR)': format_number(iqr),

        # Outliers
        'Lower Bound for Outliers': format_number(lower_bound),
        'Upper Bound for Outliers': format_number(upper_bound),
        'Number of Outliers': num_outliers,

        # Other
        'Mode': mode_value,
        'Missing Values': missing_values,
        'Unique Values': unique_values
    })


import pandas as pd

def datetime_metrics(series: pd.Series) -> pd.Series:
    """
    Calculate various metrics for a datetime series.

    Args:
        series (pd.Series): The datetime series to analyze.

    Returns:
        pd.Series: Various calculated metrics for the series.
    """
    # Convert series to datetime format, if not already
    series = pd.to_datetime(series, errors='coerce')
    
    # Calculate basic statistics
    min_date = series.min()
    max_date = series.max()
    mean_date = series.mean()
    median_date = series.median()

    # Convert datetime to string format for reporting
    min_date_str = pd.Timestamp(min_date).strftime('%Y-%m-%d') if pd.notna(min_date) else None
    max_date_str = pd.Timestamp(max_date).strftime('%Y-%m-%d') if pd.notna(max_date) else None
    mean_date_str = pd.Timestamp(mean_date).strftime('%Y-%m-%d') if pd.notna(mean_date) else None
    median_date_str = pd.Timestamp(median_date).strftime('%Y-%m-%d') if pd.notna(median_date) else None
    
    date_range = (max_date - min_date).days if pd.notna(max_date) and pd.notna(min_date) else None
    total_days = date_range  # Same as date_range in this context
    missing_values = series.isna().sum()
    unique_values = series.nunique()
    most_common_date = series.mode().values[0] if not series.mode().empty else None
    
    # Convert most_common_date to string format
    most_common_date_str = pd.Timestamp(most_common_date).strftime('%Y-%m-%d') if pd.notna(most_common_date) else None

    # Return all metrics as a pandas Series
    return pd.Series({
        'Min Date': min_date_str,
        'Max Date': max_date_str,
        'Mean Date': mean_date_str,
        'Median Date': median_date_str,
        'Date Range (Days)': date_range,
        'Total Days': total_days,
        'Missing Values': missing_values,
        'Unique Values': unique_values,
        'Most Common Date': most_common_date_str
    })

import pandas as pd

def string_metrics(series: pd.Series) -> pd.Series:
    """
    Calculate various metrics for a string series.

    Args:
        series (pd.Series): The string series to analyze.

    Returns:
        pd.Series: Various calculated metrics for the series.
    """
    # Clean the series
    series = series.fillna('')
    
    # Calculate basic string statistics
    most_frequent = series.mode()[0] if not series.mode().empty else np.nan
    unique_values = series.nunique()
    missing_values = series.isna().sum()
    empty_strings = (series == '').sum()
    
    # String length statistics
    lengths = series.apply(len)
    longest_string = lengths.max()
    shortest_string = lengths.min()
    average_length = lengths.mean()
    
    # Character frequency (top 10 characters)
    char_freq = pd.Series(''.join(series).replace(' ', '')).value_counts().head(10)
    
    # Word frequency (top 10 words, if applicable)
    words = series.str.split(expand=True).stack()
    word_freq = words.value_counts().head(10)
    
    # Return all metrics as a pandas Series
    return pd.Series({
        'Most Frequent': most_frequent,
        'Unique Values': unique_values,
        'Missing Values': missing_values,
        'Empty Strings': empty_strings,
        'Longest String Length': longest_string,
        'Shortest String Length': shortest_string,
        'Average String Length': average_length,
        # 'Top 10 Characters': char_freq.to_dict(),
        'Top 10 Words': word_freq.to_dict()
    })

import pandas as pd

def boolean_metrics(series: pd.Series) -> pd.Series:
    """
    Calculate metrics for a boolean series.

    Args:
        series (pd.Series): The boolean series to analyze.

    Returns:
        pd.Series: Various calculated metrics for the series.
    """
    return pd.Series({
        'True Count': (series == True).sum(),
        'False Count': (series == False).sum(),
        'Missing Values': series.isna().sum()
    })

import pandas as pd

def data_type_summaries(df: pd.DataFrame) -> dict:
    """
    Get comprehensive metrics for each data type in the DataFrame and return them as a dictionary of DataFrames.

    Args:
        df (pd.DataFrame): The DataFrame for which to generate the summaries.

    Returns:
        dict: A dictionary with data type as the key and DataFrame of metrics as the value.
    """
    if df is None:
        raise ValueError("DataFrame is not set.")

    # Dictionary to hold DataFrames for each data type
    summaries = {}

    # Metrics calculation functions for each data type
    metrics_funcs = {
        np.number: numeric_metrics,
        'datetime64[ns]': datetime_metrics,
        'object': string_metrics,
        'bool': boolean_metrics
    }

    # Calculate and store metrics for each data type
    for dtype, metrics_func in metrics_funcs.items():
        type_columns = df.select_dtypes(include=[dtype])
        summary_list = []

        for column in type_columns:
            metrics = metrics_func(df[column])
            metrics.name = column
            summary_list.append(pd.DataFrame(metrics).T)

        if summary_list:
            summary_df = pd.concat(summary_list)
            summary_df.reset_index(inplace=True)
            summary_df.rename(columns={'index': 'Column Name'}, inplace=True)
            summary_df = summary_df.sort_values(by='Column Name').reset_index(drop=True)
            summaries[str(dtype)] = summary_df

    # Display DataFrames in a Jupyter Notebook
    for dtype, summary_df in summaries.items():
        print(f"\nData Type: {dtype}")
        if get_ipython() is not None:  # Check if running in Jupyter Notebook
            display(summary_df)
        else:
            print(summary_df.to_string(index=False))  # Print DataFrame as string

    return summaries



#### Dataframe Correlation

In [108]:
def correlation_function(df: pd.DataFrame, dummy_variables: bool = False, dummy_threshold: int = 10, report_threshold: float = 0.3) -> pd.DataFrame:
    """
    Calculate the correlation between columns in a DataFrame, optionally creating dummy variables for categorical columns.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - dummy_variables (bool): If True, create dummy variables for string columns with unique values <= dummy_threshold.
    - dummy_threshold (int): Maximum number of unique values in a column to create dummy variables.
    - report_threshold (float): Threshold for reporting correlated pairs (both positive and negative).
    
    Returns:
    - pd.DataFrame: DataFrame with columns of the correlated pairs, their correlation values rounded to 2 decimals.
    """
    
    # Step 1: Convert string/categorical columns to dummy variables if needed
    df_copy = df.copy()
    
    if dummy_variables:
        # Identify string/categorical columns
        non_numeric_cols = df_copy.select_dtypes(include=['object', 'category']).columns
        
        for col in non_numeric_cols:
            unique_vals = df_copy[col].nunique()
            if unique_vals <= dummy_threshold:
                # Convert to dummy variables
                df_copy = pd.get_dummies(df_copy, columns=[col], drop_first=True)
    
    # Step 2: Identify numeric columns (ignoring non-numeric columns)
    numeric_df = df_copy.select_dtypes(include=[np.number])
    
    if numeric_df.empty:
        raise ValueError("No numeric columns found in the DataFrame for correlation analysis.")
    
    # Step 3: Calculate correlation matrix
    corr_matrix = numeric_df.corr().round(2)  # Round correlation values to 2 decimals

    # Step 4: Extract correlation pairs greater than the report_threshold
    corr_pairs = corr_matrix.unstack().reset_index()
    corr_pairs.columns = ['Variable1', 'Variable2', 'Correlation']
    
    # Remove self-correlations
    corr_pairs = corr_pairs[corr_pairs['Variable1'] != corr_pairs['Variable2']]
    
    # Ensure we only report each pair once (e.g., always sort Variable1 and Variable2 alphabetically)
    corr_pairs['sorted_pair'] = corr_pairs.apply(lambda x: tuple(sorted([x['Variable1'], x['Variable2']])), axis=1)
    corr_pairs = corr_pairs.drop_duplicates(subset='sorted_pair').drop(columns='sorted_pair')

    # Step 5: Filter pairs based on the report_threshold
    corr_pairs['Abs_Correlation'] = corr_pairs['Correlation'].abs()
    filtered_pairs = corr_pairs[corr_pairs['Abs_Correlation'] >= report_threshold]
    
    # Sort by absolute correlation in descending order
    filtered_pairs = filtered_pairs.sort_values(by='Abs_Correlation', ascending=False).drop(columns='Abs_Correlation')

    return filtered_pairs.reset_index(drop=True)

### Qualitative Analysis

#### Text Analysis Functions

##### Text Cleaning Functions

In [109]:
# This is a dictionary of common encoding errors

common_encoding_errors = {
    "&lt;": "<",        # Less-than sign
    "&gt;": ">",        # Greater-than sign
    "&amp;": "&",       # Ampersand
    "&quot;": "\"",     # Double quotation mark
    "&apos;": "'",      # Apostrophe
    "&nbsp;": " ",      # Non-breaking space
    "&copy;": "(c)",    # Copyright sign
    "&reg;": "(r)",     # Registered sign
    "&euro;": "EUR",    # Euro sign
    "&pound;": "GBP",   # Pound sign
    "&yen;": "Yen",     # Yen sign
    "&cent;": "c",      # Cent sign
    "&sect;": "§",      # Section sign
    "&para;": "¶",      # Pilcrow sign
    "&deg;": "°",       # Degree sign
    "&plusmn;": "±",    # Plus-minus sign
    "&micro;": "µ",     # Micro sign
    "&sup2;": "^2",     # Superscript two
    "&sup3;": "^3",     # Superscript three
    "&frac14;": "1/4",  # Fraction one quarter
    "&frac12;": "1/2",  # Fraction one half
    "&frac34;": "3/4",  # Fraction three quarters
    "&times;": "x",     # Multiplication sign
    "&divide;": "/",    # Division sign
    "&bull;": "•",      # Bullet
    "&ndash;": "-",     # En dash
    "&mdash;": "-",     # Em dash
    "&lsquo;": "'",     # Left single quotation mark
    "&rsquo;": "'",     # Right single quotation mark
    "&sbquo;": ",",     # Single low-9 quotation mark
    "&ldquo;": "\"",    # Left double quotation mark
    "&rdquo;": "\"",    # Right double quotation mark
    "&bdquo;": "\"",    # Double low-9 quotation mark
    "&hellip;": "...",  # Horizontal ellipsis
    "%20": " ",         # Space
    "%21": "!",         # Exclamation mark
    "%22": "\"",        # Double quotation mark
    "%23": "#",         # Number sign
    "%24": "$",         # Dollar sign
    "%25": "%",         # Percent sign
    "%26": "&",         # Ampersand
    "%27": "'",         # Apostrophe
    "%28": "(",         # Left parenthesis
    "%29": ")",         # Right parenthesis
    "%2A": "*",         # Asterisk
    "%2B": "+",         # Plus sign
    "%2C": ",",         # Comma
    "%2D": "-",         # Hyphen
    "%2E": ".",         # Period
    "%2F": "/",         # Slash
    "%3A": ":",         # Colon
    "%3B": ";",         # Semicolon
    "%3C": "<",         # Less-than sign
    "%3D": "=",         # Equal sign
    "%3E": ">",         # Greater-than sign
    "%3F": "?",         # Question mark
    "%40": "@",         # At sign
    "%5B": "[",         # Left square bracket
    "%5C": "\\",        # Backslash
    "%5D": "]",         # Right square bracket
    "%5E": "^",         # Caret
    "%5F": "_",         # Underscore
    "%60": "`",         # Grave accent
    "%7B": "{",         # Left curly bracket
    "%7C": "|",         # Vertical bar
    "%7D": "}",         # Right curly bracket
    "%7E": "~",         # Tilde
    "â€”": "-",         # Em dash
    "â€“": "–",         # En dash
    "â€˜": "'",         # Left single quotation mark
    "â€™": "'",         # Right single quotation mark
    "â€œ": "\"",        # Left double quotation mark
    "â€": "\"",         # Right double quotation mark
    "â€¦": "...",       # Ellipsis
    "â€": "\"",         # Right double quotation mark (alternate)
    "â€™": "'",         # Right single quotation mark (alternate)
}

In [110]:
def clean_standardize_highway_references(text):
    """Standardize highway references in the provided text."""
    # Updated pattern to capture all variations with a number
    regex_highway_search = re.compile(
        r"(?:#*\b(?:bc\s*)?(?:h(?:ighway|wy|iway|way|w|y)|hw|hwy|hway|hiway|h)\s*[\s#\.\-]*)(\d+[abcd]*)\b",
        re.IGNORECASE,
    )
    
    def replace_match(match):
        number = match.group(1)  # Capture highway number
        return f"BCHwy{number}"
    
    text = re.sub(regex_highway_search, replace_match, text)

    return text

# Main Text cleaning function
def clean_text(text):
    """
    Clean and preprocess the input text based on specified cleaning options.

    This function applies a series of text cleaning operations to the input `text` based on the 
    global `text_cleaning_options` dictionary. The cleaning operations include:
    - Converting text to lowercase
    - Expanding or contracting contractions
    - Replacing common encoding errors
    - Standardizing highway and DriveBC references
    - Using a custom search-and-replace dictionary
    - Removing stopwords
    - Stripping or replacing punctuation
    - Performing spell check
    - Stemming or lemmatizing words
    - Cleaning up extra whitespace

    Args:
        text (str): The text to be cleaned and preprocessed.

    Returns:
        str: The cleaned text after applying the specified text cleaning operations.

    Raises:
        Exception: If any error occurs during text processing, an error message is printed, 
                   and an empty string is returned.

    Notes:
        - The function uses global dictionaries for contractions, common encoding errors, 
          custom search-and-replace terms, and stopwords.
        - The `text_cleaning_options` dictionary determines which cleaning operations are applied.
        - The `ensure_string` function is assumed to ensure that `text` is a string. It is not defined here.
        - The `clean_whitespace` function is assumed to handle extra whitespace. It is not defined here.
    """


    global text_cleaning_options

    # Set Cleaning Options
    transform_contractions = text_cleaning_options["Transform contractions"]        
    convert_to_lowercase = text_cleaning_options["Convert to lowercase"]
    covert_common_encoding_errors = text_cleaning_options["Convert Common Encoding Errors"]
    
    strip_punctuation = text_cleaning_options["Strip punctuation"]
    remove_stopwords = text_cleaning_options["Remove stopwords"]

    use_custom_sar_dict = text_cleaning_options["Use custom Search-and-Replace dictionary"]
    standardize_highway_references = text_cleaning_options["Standardize highway references"]
    standardize_drivebc_references  = text_cleaning_options["Standardize drivebc references"]
    spell_check = text_cleaning_options["Spell check"]
    stem_words  = text_cleaning_options["Stem words"]
    lemmatize_words = text_cleaning_options["Lemmatize words"]

    try:
        # Make sure the data is actually text
        text = ensure_string(text,3)

        # Convert all text to lower case
        if convert_to_lowercase is True:
            text = text.lower()

        # Replace contractions
        if transform_contractions == "Expand":
            # Expand contractions
            for key, value in contractions_dict.items():
                text = text.replace(value, key)
            # Contract expansions
        elif transform_contractions == "Contract":
                for key, value in contractions_dict.items():
                    text = text.replace(key, value)

        if covert_common_encoding_errors is True:
            # Perform character replacement using the dictionary
            for improper, proper in common_encoding_errors.items():
                text = text.replace(improper, proper)

        # Standardize Highway References
        if standardize_highway_references is True:
            text = clean_standardize_highway_references(text)

        # Standardize DriveBC references
        if standardize_drivebc_references is True:
            regex_drivebc_search = re.compile(
                r"\bdrive.*bc\b",
                re.IGNORECASE,
            )
            regex_drivebc_replace = r"drivebc"
            text = re.sub(regex_drivebc_search, regex_drivebc_replace, text)

        # Use a custom search and replace dictionary
        if use_custom_sar_dict is True:
            for key, value in custom_search_and_replace_dict.items():
                text = text.replace(key, value)

        # Remove stop words
        if remove_stopwords is True:
            for key, value in external_stopword_list_dict.items():
                text = text.replace(key, value)

        # strip punctuation
        if strip_punctuation is True:
            # # This removed punctuation marks
            # text = " ".join([char for char in text if char not in string.punctuation])
            
            # This replaces punctuation marks with a space
            translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
            # Replace punctuation with spaces
            text = text.translate(translation_table)

        if spell_check is True:
            text = text.split()
            text = [spell(word) for word in text]
            text = " ".join(text)

        # Stem words
        if stem_words is True:
            text = text.split()
            text = [ps.stem(word) for word in text]
            text = " ".join(text)

        # Lemmatize words
        if lemmatize_words is True:
            text = text.split()
            text = [wn.lemmatize(word) for word in text]
            text = " ".join(text)

        # Clean Whitespace
        text = clean_whitespace(text)

        return text
    except Exception as e:
        print(f"Error processing text: {e}")
        return ""

# Add clean text to dataframe
def add_clean_text_column(df,comments):
    """
    Adds a cleaned text column to a DataFrame by applying text cleaning operations.

    This function adds a new column to the input DataFrame, `df`, which contains cleaned text
    derived from the `comments` Series. The cleaning is performed using the `clean_text` function.

    Args:
        df (pd.DataFrame): The DataFrame to which the cleaned text column will be added.
        comments (pd.Series): A Series containing the text data to be cleaned and added to the DataFrame.

    Returns:
        pd.DataFrame: The original DataFrame with an additional column named 'clean_text' that contains the cleaned text.

    Raises:
        ValueError: If `comments` is not a Series or if its length does not match the number of rows in `df`.
        Exception: If any error occurs during the application of the `clean_text` function.

    Notes:
        - The `comments` Series should have the same length as the number of rows in the DataFrame `df`.
        - The `clean_text` function is expected to be defined elsewhere and handle the text cleaning process.
    """
    if not isinstance(comments, pd.Series):
        raise ValueError("The 'comments' argument must be a pandas Series.")
    if len(comments) != len(df):
        raise ValueError("The length of 'comments' must match the number of rows in the DataFrame.")

    try:
        df["clean_text"] = comments.apply(lambda x: clean_text(x))
        return df
    except Exception as e:
        print(f"Error adding clean text column: {e}")
        raise

##### N-Gram Analysis

In [111]:
def ngram_counter(series):
    """
    Counts and returns the most common n-grams in a pandas Series of text data.

    The function prompts the user to input the starting and ending n-gram lengths, 
    as well as the number of top n-grams to find for each length. It then processes 
    the text data, tokenizes it, generates n-grams for the specified lengths, and 
    counts their occurrences. Finally, it returns a DataFrame with the n-grams, 
    their types, and counts.

    Parameters:
    series (pandas.Series): A pandas Series containing text data.

    Returns:
    pandas.DataFrame: A DataFrame with columns 'N-Gram', 'N-Gram Type', and 'Count',
                      containing the most common n-grams and their counts for each
                      specified n-gram length.

    User Inputs:
    - Starting n-gram length: An integer specifying the smallest n-gram length (e.g., 
                              1 for unigram, 2 for bigram, etc.).
    - Ending n-gram length: An integer specifying the largest n-gram length.
    - Number of n-grams: An integer specifying the number of top n-grams to find 
                         for each n-gram length.

    Example:
    >>> import pandas as pd
    >>> series = pd.Series(["This is a sample text.", "This text is for testing purposes."])
    >>> result = ngram_counter(series)
    >>> print(result)
               N-Gram N-Gram Type  Count
    0            this       1-gram      2
    1              is       1-gram      2
    2              a       1-gram      1
    3          sample       1-gram      1
    4            text       1-gram      2
    5           this is     2-gram      2
    6              is a     2-gram      1
    7          sample text  2-gram      1
    8            text is    2-gram      1
    9              is for   2-gram      1
    10           for testing 2-gram     1
    11 testing purposes 2-gram  1
    """
    # Prompt user for the starting and ending n-gram lengths
    os.system("cls")
    while True:
        try:
            start_length = int(input("Enter the starting n-gram length (1 for unigram, 2 for bigram, etc.): "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    
    while True:
        try:
            end_length = int(input("Enter the ending n-gram length: "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    # Prompt user for the number of n-grams to find
    while True:
        try:
            num_ngrams = int(input("Enter the number of n-grams to find for each type: "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    ngrams_list = []

    # Process each n-gram length
    for n in range(start_length, end_length + 1):
        ngram_counts = Counter()
        
        # Process each text in the series
        for text in series.dropna().str.lower():
            tokens = word_tokenize(text)
            if len(tokens) >= n:
                # Generate and count n-grams directly
                generated_ngrams = ngrams(tokens, n)
                ngram_counts.update(generated_ngrams)

        # Get the top n-grams of each type
        top_ngrams = ngram_counts.most_common(num_ngrams)
        for ngram, count in top_ngrams:
            ngrams_list.append([' '.join(ngram), f"{n}-gram", count])

    # Create a DataFrame with the n-grams, their types, and counts
    result_df = pd.DataFrame(ngrams_list, columns=['N-Gram', 'N-Gram Type', 'Count'])

    return result_df

##### Isolate Parts of Speech Functions

In [112]:
def isolate_pos(series, selected_parts_of_speech, string_length_threshold):
    """
    Isolates and counts words from a text series based on selected parts of speech and a string length threshold.

    This function processes the input `series` by tokenizing it, tagging the tokens with their parts of speech,
    and filtering the words based on the selected parts of speech. It then removes stop words, applies a string 
    length threshold, and returns a DataFrame with word counts for the remaining words.

    Args:
        series (pd.Series or str): The input text data, either as a pandas Series or a single string. 
        selected_parts_of_speech (list of str): A list of parts of speech to filter the tokens. 
        string_length_threshold (int): The minimum length of words to include in the final word count.

    Returns:
        pd.DataFrame: A DataFrame containing two columns:
            - 'word': The words filtered by the selected parts of speech and length threshold.
            - 'count': The frequency of each word.

    Raises:
        ValueError: If `selected_parts_of_speech` is empty or `string_length_threshold` is not a positive integer.
        TypeError: If `series` is neither a pandas Series nor a string.

    Notes:
        - The `series` is converted to a string if it is a pandas Series.
        - NLTK's `word_tokenize` and `pos_tag` functions are used for tokenization and part-of-speech tagging.
        - The function uses a predefined list of stop words to filter out common, non-informative words.
        - Words are filtered based on the specified length threshold before counting their occurrences.

    Example:
        >>> df = pd.Series(["The quick brown fox jumps over the lazy dog."])
        >>> pos_list = ['NN', 'JJ']  # Example parts of speech: Noun and Adjective
        >>> threshold = 3
        >>> isolate_pos(df, pos_list, threshold)
           word  count
        0   quick      1
        1   brown      1
        2   jumps      1
        3   over       1
    """


    if not selected_parts_of_speech:
        print("No Parts of Speech have been selected. Please select them and try again.")
        input(f"Press 'Enter' to continue.")
        return

    if isinstance(series, pd.Series):
        series = series.to_string()

    # Tokenize the sentence into individual words
    tokens = nltk.word_tokenize(series)

    # Use NLTK's POS tagger to tag each word with its part of speech
    pos_tags = nltk.pos_tag(tokens)

    # Create a list of words filtered by chosen Part-of-Speech (PoS) type
    word_list = []
    print("Tagging parts of speech...")
    for word, pos in pos_tags:
        if pos in selected_parts_of_speech:
            word_list.append(word)

    # Remove any stop words from the list
    stop_words = [
        "..", "'m", "'s", "#name?", "are", "be", "been", "being", "did", "do", "does", "done",
        "get", "go", "had", "has", "have", "i", "ì", "is", "make", "n/a", "needs", "put", "Q",
        "seems", "take", "use", "vs", "was", "way", "were"
    ]
    
    cleaned_word_list = [element for element in word_list if element not in stop_words]

    # Apply string length threshold
    filtered_word_list = [word for word in cleaned_word_list if len(word) >= string_length_threshold]

    # Count the frequency of each word
    word_counts = pd.Series(filtered_word_list).value_counts().reset_index()

    # Rename the columns of the DataFrame
    word_counts.columns = ["word", "count"]

    # Sort the DataFrame by count in descending order
    word_counts = word_counts.sort_values(by="count", ascending=False).reset_index(drop=True)

    return word_counts

##### Sentiment Analysis

In [113]:
def add_sentiment_column(df, text_column):
    """
    Add a sentiment score column to the DataFrame based on the specified text column using VADER.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    text_column (str): The name of the column containing the text for sentiment analysis.

    Returns:
    pd.DataFrame: The DataFrame with an added "sentiment" score column.
    """
    # Ensure the text_column exists in the DataFrame
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in DataFrame columns.")

    # Initialize the VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Define a function to calculate the sentiment score
    def get_sentiment(text):
        if pd.isnull(text) or not isinstance(text, str) or text.strip() == "":
            return None  # Return None for rows without valid text
        sentiment = analyzer.polarity_scores(text)
        return sentiment['compound']  # Compound sentiment score

    # Apply the sentiment function to the text column and add the results to a new column
    df['sentiment'] = df[text_column].apply(get_sentiment)

    return df

# Read Data

In [ ]:
# Enter the file (or folder) with the data to load into a dataframe
# read_file_path = r'C:\Users\Windows\Downloads\archive (7)\netflix1.csv'
# read_file_path = r'C:\Users\Windows\Downloads\happyscore_income.csv'

read_file_path = r'D:\Data Analysis\Qualitative Analysis\comments_for_demo.csv'

df = load_to_dataframe(read_file_path)

## Data Loading Notes

### Read a Single File

Force a file to load as text with force_plain_text=True </br>
When loading an Excel file, use sheet_name="Sheet1" to specify the sheet name to load </br>

 - df = load_to_dataframe("path/to/file.csv", file_type="csv")
 - df = load_to_dataframe("path/to/file.txt", file_type="txt", force_plain_text=True)
 - df = load_to_dataframe("path/to/file.xlsx", file_type="xlsx", sheet_name="Sheet1")
 - df = load_to_dataframe("path/to/file.json", file_type="json")
 - df = load_to_dataframe("path/to/file.parquet", file_type="parquet")
 - df = load_to_dataframe("path/to/file.hdf", file_type="hdf")
 - df = load_to_dataframe("path/to/file.feather", file_type="feather")

### Read Multiple Files
 - df = load_to_dataframe("path/to/folder", file_type="csv")
 - df = load_to_dataframe("path/to/folder", file_type="log", force_plain_text=True)

# General Analysis

## Dataframe Diagnostics

In [ ]:
column_list = df.columns.tolist()
print("Columns:")
print("========")
for column_name in column_list:
    print(column_name)

diags = dataframe_diagnostics(df)

## Data Summaries

### Dataframe Summary

In [ ]:
summary_df = overall_summary(df)
display(summary_df)

In [ ]:
# print(type(df.columns))

index = df.columns
# Convert to list and join with a separator, like a newline or comma
# Join the index into a user-friendly string
user_friendly_output = "\n".join(index.tolist())

print("Index Columns:\n", user_friendly_output)

In [ ]:
display(summary_df)

### Column Summaries

In [ ]:
# Get the summary
summary_df = data_types_summary(df)
display(summary_df)

### Column Details

In [ ]:
# Run the function
summaries = data_type_summaries(df)
display(summaries)

## Correlation

In [ ]:
result = correlation_function(df, dummy_variables=True, dummy_threshold=5, report_threshold=.5)
display(result)

# Qualitiative Analysis

## Set Data Series from Dataframe for Analysis

In [129]:
text_for_analysis = df["Comment"]


## Load External Dictionaries

In [89]:
# Load Custom Dictionaries
external_stopword_list_dict = load_csv_to_dict(external_stopword_list, "stopword", "replace")
contractions_dict = load_csv_to_dict(external_contractions_dictionary, "expansion", "contraction")
custom_search_and_replace_dict = load_csv_to_dict(custom_search_and_replace_dictionary, "search_term", "replace_term")

## Text Analysis

### Text Cleaning

In [90]:
text_cleaning_options = {
    "Convert to lowercase": True,
    "Convert Common Encoding Errors": True,
    "Strip punctuation": True,  
    "Remove stopwords": False,  # Examples could include: a, about, at, be, by
    "Transform contractions": "Expand", 
    "Use custom Search-and-Replace dictionary": True,
    "Standardize highway references": True, # 
    "Standardize drivebc references": True,
    "Spell check": False,
    "Stem words": False,
    "Lemmatize words": False,
}


In [ ]:
cleaned_text = add_clean_text_column(df, text_for_analysis)
cleaned_comments = cleaned_text["clean_text"]

base_cleaned_text_report = r"D:\Data Analysis\Qualitative Analysis\reports\cleaned_text_report.csv"
cleaned_text_report = save_dataframe_with_incremented_filename(base_cleaned_text_report)

# remove_columns = ["ResponseID", "Question Text", "Question Number"]

# # Check which columns in the list are present in the DataFrame
# columns_present = [col for col in remove_columns if col in cleaned_text.columns]

# # Drop the columns that are present
# df.drop(columns=columns_present, inplace=True)

# print(cleaned_text)
cleaned_text.to_csv(cleaned_text_report,index=False)
open_in_excel(cleaned_text_report)

### N-Grams

In [ ]:
# NGrams - Cleaned
base_ngram_report_cleaned = r"D:\Data Analysis\Qualitative Analysis\reports\ngram_report_cleaned.csv"
return_ngrams = ngram_counter(cleaned_comments)
ngram_report = save_dataframe_with_incremented_filename(base_ngram_report_cleaned)
return_ngrams.to_csv(ngram_report,index=False)
open_in_excel(ngram_report)

### Parts of Speech

In [93]:
all_verbs = ["VB","VBD","VBG","VBN","VBP","VBZ"]
all_nouns = ["NN","NNP","NNS"]
all_nouns_and_verbs = ["NN","NNP","NNS","VB","VBD","VBG","VBN","VBP","VBZ"]

parts_of_speech = [
    ("CC", "conjunction, coordinating"),
    ("CD", "numeral, cardinal"),
    ("DT", "determiner"),
    ("EX", "existential there"),
    ("IN", "preposition or conjunction, subordinating"),
    ("JJ", "adjective or numeral, ordinal"),
    ("JJR", "adjective, comparative"),
    ("JJS", "adjective, superlative"),
    ("LS", "list item marker"),
    ("MD", "modal auxiliary"),
    ("NN", "noun, common, singular or mass"),
    ("NNP", "noun, proper, singular"),
    ("NNS", "noun, common, plural"),
    ("PDT", "pre-determiner"),
    ("POS", "genitive marker"),
    ("PRP", "pronoun, personal"),
    ("RB", "adverb"),
    ("RBR", "adverb, comparative"),
    ("RBS", "adverb, superlative"),
    ("RP", "particle"),
    ("TO", "to as preposition or infinitive marker"),
    ("UH", "interjection"),
    ("VB", "verb, base form"),
    ("VBD", "verb, past tense"),
    ("VBG", "verb, present participle or gerund"),
    ("VBN", "verb, past participle"),
    ("VBP", "verb, present tense, not 3rd person singular"),
    ("VBZ", "verb, present tense, 3rd person singular"),
    ("WDT", "WH-determiner"),
    ("WP", "WH-pronoun"),
    ("WRB", "Wh-adverb")
]

In [ ]:
pos_df = isolate_pos(cleaned_comments, all_nouns_and_verbs, 3)

# Specify the length threshold
string_length_threshold = 3

# Filter out rows where the length of the string in the "word" column is less than the threshold
filtered_pos_df = pos_df[pos_df['word'].apply(lambda x: len(x) >= string_length_threshold)]

base_pos_text_report = r"D:\Data Analysis\Qualitative Analysis\reports\parts_of_speech_text_report.csv"
pos_text_report = save_dataframe_with_incremented_filename(base_pos_text_report)

filtered_pos_df.to_csv(pos_text_report,index=False)
open_in_excel(pos_text_report)   

### Sentiment Analysis

In [ ]:
# Add sentiment column to the DataFrame Use the text name for the header column, not a pandas series
df_with_sentiment = add_sentiment_column(df, "Comment")

base_sentiment_report = r"D:\Data Analysis\Qualitative Analysis\reports\sentiment_report.csv"
sentiment_report = save_dataframe_with_incremented_filename(base_sentiment_report)

df_with_sentiment.to_csv(sentiment_report,index=False)
open_in_excel(sentiment_report)